<a href="https://colab.research.google.com/github/OlegKuzmin1968/Oleg_Kuzmin/blob/main/%D0%A4%D0%BE%D1%80%D0%BC%D0%B8%D1%80%D0%BE%D0%B2%D0%B0%D0%BD%D0%B8%D0%B5_x_train_%D0%B4%D0%BB%D1%8F_2D_%D0%BC%D0%BE%D0%B4%D0%B5%D0%BB%D0%B8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


#*Формирование x_train для 2D модели. ЗАДАЧА*

Работаем с выборкой из 14 пациентов начиная с 303 и до 396 (только по тем для которых сделана сегментация преврального выпота).

Цель:

    1. Написать код по преобразованию DICOM файла в np-массив для каждого пациента.

    2. Предусмотреть возможность сохранения полученного np-массив с соответствующим идентификатором пациента на googl Disk

    3. Написать генератор данных, который использую функцию преобразования массива сможет подавать эти данные в НС
    
    ...для каждого пациента  из слайсов независимо от пациентов, при этом каждому слайсу присвоить двузначный правильный ответ: имеется ли на этом слайсе попаженная часть (выпот).x_train

# Загрузки

In [ ]:
from nibabel.viewers import OrthoSlicer3D
from nibabel import nifti1
import nibabel as nib
from matplotlib import pylab as plt
import matplotlib
import numpy as np
from numpy.core.multiarray import dtype
from matplotlib import pyplot, cm
import os
from PIL import Image # Синтаксис и описание функции Image.open():
import random

In [ ]:
#!rm -R /content/drive/MyDrive/Baza/Stag_1/Effusions_Directory/LUNG1-420 # УДАЛЕНИЕ НЕПУСТОЙ ПАПКИ В COLABE

rm: cannot remove '/content/drive/MyDrive/Baza/Stag_1/Effusions_Directory/LUNG1-420': No such file or directory


In [ ]:
#!rm -R /content/drive/MyDrive/Baza/Stag_1/np

https://drive.google.com/drive/folders/1QHsJbmCAFGp6Hl-tgtHutvToTy2I1Lj_?usp=drive_link

**Дмитрий Черемисов** Сюда загрузил первый проект и датасет.

**Из NB "Формирование x_train и y_train на основе 2D массива", берем список test-овой выборки соответствующих пациентов.**

In [ ]:
keys_list_134 = ['LUNG1-303', 'LUNG1-307', 'LUNG1-312', 'LUNG1-325', 'LUNG1-331', 'LUNG1-336', 'LUNG1-361', 'LUNG1-362', 'LUNG1-366', 'LUNG1-367', 'LUNG1-376', 'LUNG1-377', 'LUNG1-381', 'LUNG1-396']


In [ ]:
len(keys_list_134)

14


# Работа с оригинальным изображением

### Загрузки библиотек для работы с DICOM

In [ ]:
!pip install pydicom

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 20.7 MB/s eta 0:00:00


In [ ]:
# Единственный интересный момент здесь заключается в том, как я упоминал в разделе "Пакет pydicom",
# что пакет pydicom импортируется как dicom, поэтому будьте осторожны с этим.

import pydicom           # запустился так
import pydicom.data      # + metod

## **Путь к локальной папке оригинальных снимков для test**

In [144]:
# Присвоим переменной q путь к папке где хранятся все оригинальные снимки DICOM
q = '/content/drive/MyDrive/Baza/Stag_1/Original_Directory'
print (' q-', q,'\n','Тип переменной q -', type(q))

 q- /content/drive/MyDrive/Baza/Stag_1/Original_Directory 
 Тип переменной q - <class 'str'>


In [146]:
# Путь к оригинальным снимкам тестовой выборки на моем Googl Disk
Original_Directory = '/content/drive/MyDrive/Baza/Stag_1/Original_Directory/Test'

Original_list = os.listdir(Original_Directory)   # занесем всех пациентов с оригинальными снимками в список
Original_list.sort()                             # отсортируем список
Original_list.pop(0)                             # удалим первый элемент

print('Сам список после сортировки: ', Original_list)
pac = len(Original_list)                         # количество пациентов с оригинальными снимками в тестовой выборке DS
print('Количество пациентов-', pac)


Сам список после сортировки:  ['LUNG1-381', 'LUNG1-396']
Количество пациентов- 2


## Функция составления списка слайсов

In [ ]:
def LUNG_n_slise (n, Original_list):

  ''' Функция составления списка слайсов оригинального снимка
      Вход:  n - порядковый номер пациента в списке
             Original_list - список пациентов с оригинальными списками
      Выход: Список слайсов в оригинальном снимке одного пациента '''

  lun_1 = Original_list [n]
  #print(Original_list [n],'n')
  print('   Файл со снимком одного из пациентов',lun_1)
  Original_list2 = os.listdir('/content/drive/MyDrive/Baza/Stag_1/Original_Directory/Test'+'/' +lun_1)
  lun_2 = Original_list2 [0]
  #print('lun_2',lun_2)
  Original_list3 = os.listdir('/content/drive/MyDrive/Baza/Stag_1/Original_Directory/Test'+'/'+ lun_1 + '/' + lun_2)
  lun_3 = Original_list3 [0]
  #print('lun_3',lun_3)
  base = '/content/drive/MyDrive/Baza/Stag_1/Original_Directory/Test'+'/'+ lun_1 + '/' + lun_2 + '/' + lun_3
  Original_list4 = os.listdir('/content/drive/MyDrive/Baza/Stag_1/Original_Directory/Test'+'/'+ lun_1 + '/' + lun_2 + '/' + lun_3)
  Original_list4.sort()
  #k = str('lung_' + str(n))
  #print(len(Original_list4))
  #print('Original_list4', Original_list4 [12])
  return Original_list4, base


In [ ]:
LUNG_n_slise (0,Original_list)

   Файл со снимком одного из пациентов LUNG1_396


(['1-001.dcm',
  '1-002.dcm',
  '1-003.dcm',
  '1-004.dcm',
  '1-005.dcm',
  '1-006.dcm',
  '1-007.dcm',
  '1-008.dcm',
  '1-009.dcm',
  '1-010.dcm',
  '1-011.dcm',
  '1-012.dcm',
  '1-013.dcm',
  '1-014.dcm',
  '1-015.dcm',
  '1-016.dcm',
  '1-017.dcm',
  '1-018.dcm',
  '1-019.dcm',
  '1-020.dcm',
  '1-021.dcm',
  '1-022.dcm',
  '1-023.dcm',
  '1-024.dcm',
  '1-025.dcm',
  '1-026.dcm',
  '1-027.dcm',
  '1-028.dcm',
  '1-029.dcm',
  '1-030.dcm',
  '1-031.dcm',
  '1-032.dcm',
  '1-033.dcm',
  '1-034.dcm',
  '1-035.dcm',
  '1-036.dcm',
  '1-037.dcm',
  '1-038.dcm',
  '1-039.dcm',
  '1-040.dcm',
  '1-041.dcm',
  '1-042.dcm',
  '1-043.dcm',
  '1-044.dcm',
  '1-045.dcm',
  '1-046.dcm',
  '1-047.dcm',
  '1-048.dcm',
  '1-049.dcm',
  '1-050.dcm',
  '1-051.dcm',
  '1-052.dcm',
  '1-053.dcm',
  '1-054.dcm',
  '1-055.dcm',
  '1-056.dcm',
  '1-057.dcm',
  '1-058.dcm',
  '1-059.dcm',
  '1-060.dcm',
  '1-061.dcm',
  '1-062.dcm',
  '1-063.dcm',
  '1-064.dcm',
  '1-065.dcm',
  '1-066.dcm',
  '1-067.d

## Создание np-файлов для 2D модели test


### Функция создания словаря путей к слайсам тестовой выборки

In [109]:
counter = 0             # сквозной счетчик слайсов тестовой выборки
orig_dist = {}          # словарь- сквозной порядковый номер слайса:путь к слайсу конкретного пациента

for pa in range (pac):  # цикл обращения ко всем папкам пациентов

  pass_dicom_list, base = LUNG_n_slise (pa,Original_list)              # список слайсов в оригинальном снимке пациента, путь к папке пациента
  #print('Количество слайсов в файле пациента:', len(pass_dicom_list))
  #print('Название очередного слайса из списка:', pass_dicom_list[pa])
  #print('Путь к слайсам конкретного пациента -', base)

  for slis in range (len(pass_dicom_list)):  # цикл работы со слайсом очередного пациента
    x_pa = base + '/' + pass_dicom_list[slis]    # собираем полный путь к файлу-слайсу очередного пациента
    print('x_pa',x_pa)
    print('Сквозной порядковый номер слайса -', counter)

    orig_dist[counter] = x_pa                    # добавляем в словарь новую пару
    counter += 1                                 # изменяем значение счетчика на 1

print('все, словарь для x_train готов')



   Файл со снимком одного из пациентов LUNG1-381
x_pa /content/drive/MyDrive/Baza/Stag_1/Original_Directory/Test/LUNG1-381/03-20-2010-NA-NA-60601/0.000000-NA-45095/1-001.dcm
Сквозной порядковый номер слайса - 0
x_pa /content/drive/MyDrive/Baza/Stag_1/Original_Directory/Test/LUNG1-381/03-20-2010-NA-NA-60601/0.000000-NA-45095/1-002.dcm
Сквозной порядковый номер слайса - 1
x_pa /content/drive/MyDrive/Baza/Stag_1/Original_Directory/Test/LUNG1-381/03-20-2010-NA-NA-60601/0.000000-NA-45095/1-003.dcm
Сквозной порядковый номер слайса - 2
x_pa /content/drive/MyDrive/Baza/Stag_1/Original_Directory/Test/LUNG1-381/03-20-2010-NA-NA-60601/0.000000-NA-45095/1-004.dcm
Сквозной порядковый номер слайса - 3
x_pa /content/drive/MyDrive/Baza/Stag_1/Original_Directory/Test/LUNG1-381/03-20-2010-NA-NA-60601/0.000000-NA-45095/1-005.dcm
Сквозной порядковый номер слайса - 4
x_pa /content/drive/MyDrive/Baza/Stag_1/Original_Directory/Test/LUNG1-381/03-20-2010-NA-NA-60601/0.000000-NA-45095/1-006.dcm
Сквозной порядко

In [111]:
orig_dist[164] # смотрим содержание словаря

'/content/drive/MyDrive/Baza/Stag_1/Original_Directory/Test/LUNG1-396/05-01-2010-NA-NA-28227/0.000000-NA-05554/1-031.dcm'

### Функция y_gen генератор


In [ ]:
# Чтение массива y_train из googl diska в настоящий NB_colab

with open('/content/drive/MyDrive/Baza/Stag_1/Effusions_np/y_train_1876.txt', 'r') as f:
  y_train_1876 = np.loadtxt(f)
  print('y_train_1876', y_train_1876, type(y_train_1876), y_train_1876.shape)

y_train_1876 [0. 0. 0. ... 0. 0. 0.] <class 'numpy.ndarray'> (1876,)


In [ ]:
y_train_1876 [:200]  # вся последовательность представляет собой, с определенной долей приближения, гармоничный ряд.

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.])

In [139]:
def batch_y (y_train):            # на вход функции подаем список слайсов обучающей выборки  с параметром sufle
  for i in range(len(y_train)):   # длина цикла определяется размером количеством слайсов обучающей выборки
    yield y_train [i]             # при очередном обращении выдает соответствующее значение y_train

# На входе у нас есть список y_train_1876 содержащий правильные ответы для каждого слайса из test выборки

y_gen = batch_y (y_train_1876)    # функцию: <class 'function'> batch_y - превращаем в генератор y_gen <generator object batch_it at 0x7f3ea189f990>

In [140]:
print('1-е обращение', next (y_gen))
print('2-е обращение', next (y_gen))
print('3-е обращение', next (y_gen))

1-е обращение 0.0
2-е обращение 0.0
3-е обращение 0.0


In [141]:
for k in range (30):
  print(k,'-е','обращение', next (y_gen))

0 -е обращение 0.0
1 -е обращение 0.0
2 -е обращение 0.0
3 -е обращение 0.0
4 -е обращение 0.0
5 -е обращение 0.0
6 -е обращение 0.0
7 -е обращение 0.0
8 -е обращение 0.0
9 -е обращение 0.0
10 -е обращение 0.0
11 -е обращение 0.0
12 -е обращение 0.0
13 -е обращение 0.0
14 -е обращение 0.0
15 -е обращение 0.0
16 -е обращение 0.0
17 -е обращение 0.0
18 -е обращение 0.0
19 -е обращение 0.0
20 -е обращение 0.0
21 -е обращение 1.0
22 -е обращение 1.0
23 -е обращение 1.0
24 -е обращение 1.0
25 -е обращение 1.0
26 -е обращение 1.0
27 -е обращение 1.0
28 -е обращение 1.0
29 -е обращение 1.0


In [38]:
next (y_gen)

1.0

### Функция x_gen через словарь

In [132]:
len_dist = list(orig_dist.keys())   # все ключи словаря перевели в список ключей
# print (len_dist[90:100])
# len(len_dist)                     # теперь можем посмотреть размер словаря (через дляну списка)

def batch_x (orig_dist): # на вход функции подаем список слайсов обучающей выборки  с параметром sufle
  '''
     вход: orig_dist- словарь: сквозной порядковый номер слайса:путь к слайсу конкретного пациента
     выход: np_orig- слайс DICOM преобразованый в np массив размерностью (512,512,1)

  '''
  for slis in range (len(len_dist)-1):  # цикл работы со слайсами ограничен длиной словаря -1
    q = orig_dist[slis]
    print('q', q)
    ds = pydicom.dcmread(q)                              # Функция считывает одно изображение и возвращает объект pydicom.dataset.FileDataset
    test_str = ds.PatientID                              # читаем из файла ID пациента
    print('Patient', test_str, 'Slis', slis)
    np_orig = np.array(ds.pixel_array, dtype= 'int16')   # можно изменить размер переменной
    print('np_orig.shape', np_orig.shape)
    q_np = np.reshape (np_orig,[512,512,1])              # Добавим размерность для подачи в Conv2D слой НС (принимает только 3-х мерный массив)

    print('все')

    yield np_orig


In [130]:
x_gen = batch_x (orig_dist)   # функцию: <class 'function'> batch_x - превращаем в генератор x_gen <generator object batch_it at 0x7f3ea189f990>

In [131]:
print('1-е обращение', next (x_gen))
print('2-е обращение', next (x_gen))
print('3-е обращение', next (x_gen))

q /content/drive/MyDrive/Baza/Stag_1/Original_Directory/Test/LUNG1-381/03-20-2010-NA-NA-60601/0.000000-NA-45095/1-001.dcm
Patient LUNG1-381 Slis 0
np_orig.shape (512, 512)
все
1-е обращение [[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]
q /content/drive/MyDrive/Baza/Stag_1/Original_Directory/Test/LUNG1-381/03-20-2010-NA-NA-60601/0.000000-NA-45095/1-002.dcm
Patient LUNG1-381 Slis 1
np_orig.shape (512, 512)
все
2-е обращение [[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]
q /content/drive/MyDrive/Baza/Stag_1/Original_Directory/Test/LUNG1-381/03-20-2010-NA-NA-60601/0.000000-NA-45095/1-003.dcm
Patient LUNG1-381 Slis 2
np_orig.shape (512, 512)
все
3-е обращение [[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]


## Генератор batch_list негармоничной последовательности

формируем список случайных номеров np_слайсов

**Не хотелось бы, чтобы НС восприняла эту гармоничную последовательность как определенную закономерность, поэтому перемешаю слайсы в x_train y_train случайным образом**

In [ ]:
# Создадим список последовательности простых чисел от 0 до числа характеризующего длину массива y_train
test_list = []
for x in range(y_train_1876.shape[0]):
  test_list.append(x)

print(len(test_list))
print(test_list [:10],test_list [-10:])


1876
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9] [1866, 1867, 1868, 1869, 1870, 1871, 1872, 1873, 1874, 1875]


In [ ]:
O = list(range(1876))
print(type(O))
O[1532:1538]

<class 'list'>


[1532, 1533, 1534, 1535, 1536, 1537]

#### rand_list

In [ ]:
# формирование списка батчей для обучения НС из test выборки
rand_list = []  # список для хранения случайной последовательности подаваемых в НС слайсов
#for nb in range(n_batch):
  #print('nb', nb)
  #nb_list = []
for b in range(len(test_list)):             # цикл случайной последовательности
    #print(b)
  rand = random.randint(0,len(test_list)-1) # генерируем случайное число в диапазоне о - len(test_list)
    #print('rand', rand)
    #print('вне', len(test_list))
  sl = test_list [rand]                     # достаем из тестовой выборки порядковый номер слайса
  rand_list.append(sl)                        # добавляем извлеченный порядковый номер слайса в списокфайлов очередного батча
    #print(nb_list)
  test_list.remove (sl)                     # убираем из test_list порядковый номер только что внесенного в список очередного ватча слайс
    #print('Удалили sl', sl)

print(rand_list [100:120])                  # запишем в batch_list очередной батч с batch_size-номерами слайсов из test выборки

print('len(rand_list)', len(rand_list))

[782, 1356, 854, 1845, 148, 1501, 1375, 1819, 694, 1517, 1547, 200, 1170, 18, 1497, 738, 1619, 1298, 1679, 942]
len(rand_list) 1876


In [ ]:
# Исходной информацией для этого блока является длинна списка y_train - кол-во слайсов в testвыборке
# Глобальным параметром для формирования batch_list является размер batch_size при обучении сети
# используем метод - random.choince — возвращает случайный элемент из любой последовательности (строки, списка, кортежа):

batch_size = 5
n_batch = round(len(test_list) / batch_size)
print('Расчитали количество n_batch-',n_batch, 'при обучении НС при размере batch_size-', batch_size)


Расчитали количество n_batch- 375 при обучении НС при размере batch_size- 5


In [ ]:
# Создадим список последовательности простых чисел от 0 до числа характеризующего длину массива y_train
test_list = []                               # список для последовательности простых чисел
for x in range(y_train_1876.shape[0]):       # цикл генерации простых чисел в диапазоне
  test_list.append(x)                        # добавляем очередное число в список

#print(len(test_list))
#print(test_list[-1])

# формирование списка батчей для обучения НС из test выборки
batch_list = []  # список для хранения списков batch из случайных значений размера test выборки
for nb in range(n_batch):
  #print('nb', nb)
  nb_list = []
  for b in range(batch_size):                 # цикл создания списка слайсов одного ватча
    #print(b)
    rand = random.randint(0,len(test_list)-1) # генерируем случайное число в диапазоне о - len(test_list)
    #print('rand', rand)
    #print('вне', len(test_list))
    sl = test_list [rand]                     # достаем из тестовой выборки порядковый номер слайса
    nb_list.append(sl)                        # добавляем извлеченный порядковый номер слайса в списокфайлов очередного батча
    #print(nb_list)
    test_list.remove (sl)                     # убираем из test_list порядковый номер только что внесенного в список очередного ватча слайс
    #print('Удалили sl', sl)

  batch_list.append(nb_list)                  # запишем в batch_list очередной батч с batch_size-номерами слайсов из test выборки

print('len(batch_list)', len(batch_list))

len(batch_list) 375


In [ ]:
batch_list [:3]  # срез списка батчей вызова слайсов test выборки

[[1691, 1113, 1312, 1594, 808],
 [1112, 359, 120, 352, 976],
 [616, 1159, 315, 1181, 1604]]

## test


**Запись массива в файл 2D**

In [ ]:
# Модель создания пути для записи массива в файл
a = Original_list [0]                            # берем из списка пациентов один ID
b = '/content/drive/MyDrive/Baza/Stag_1/np/'     # указываем путь к папке, где будем хранить массивы
c = b + a                                        # собираем в одку конструкцию путь и имя файле в который надо писать массив
c                                                # смотрим получившуюся конструкцию

mas = np.zeros ((12,12), dtype='int16')          # создадим пустой 2D массив
with open(c, 'w') as form:                       # Оператор with создает диспетчер контекста в Пайтоне,
                                                 # который автоматически закрывает файл для вас,
                                                 # по окончанию работы в нем.
  np.savetxt(form, mas)                          # записываем файл на диск

**Чтение массива 2D из файла**

In [ ]:
# Чтения массива в NB colab d gthtvtyye. np_array:

with open('/content/drive/MyDrive/Baza/Stag_1/np/LUNG1_090', 'r') as f:
  po = np.loadtxt(f)
  print(type(po), po.shape)

<class 'numpy.ndarray'> (125, 262144)


**Преобразование скаченного из файла массива 2D в 3D-массив**

In [ ]:
from numpy.core.multiarray import dtype
LUNG1_090_3D = np.reshape(po,(125,512,512)) # сделаем обратное преобразование размерности 2D в 3D
print('Восстановленный D3', LUNG1_090_3D.shape)

Восстановленный D3 (125, 512, 512)


In [ ]:
numpy_value = np.uint16(LUNG1_090_3D) # Преобразуем элементы переменной LUNG1_090_3D к типу np.uint16

In [ ]:
print(numpy_value [64,250:260,250:260])
print(type(numpy_value))
print(numpy_value.dtype) # Выводим на экран тип элементов массива numpy_digital

[[ 216  185  163   99  128  338  885 1020 1026  996]
 [ 187  154  172  120  121  376  852  966 1000 1010]
 [ 146  127  147  118  125  402  847  968  998 1044]
 [ 136  122  160  109  162  452  920  973 1013 1029]
 [ 108  108  161  142  204  606 1003 1039 1081 1067]
 [ 200  228  190  206  417  911 1046 1064 1089 1122]
 [ 230  217  165  227  617 1055 1067 1066 1080 1099]
 [ 238  189  165  308  842 1103 1084 1084 1087 1101]
 [ 182  156  200  528  966 1082 1076 1099 1092 1062]
 [ 241  238  487  818 1011 1050 1091 1103 1090 1069]]
<class 'numpy.ndarray'>
uint16


In [ ]:
print(LUNG1_090_3D [64,250:260,250:260])
print(type(LUNG1_090_3D))          # Выводим на экран тип переменной LUNG1_090_3D
print(LUNG1_090_3D.dtype)          # Выводим на экран тип элементов (размер в байтах) массива LUNG1_090_3D

[[ 216.  185.  163.   99.  128.  338.  885. 1020. 1026.  996.]
 [ 187.  154.  172.  120.  121.  376.  852.  966. 1000. 1010.]
 [ 146.  127.  147.  118.  125.  402.  847.  968.  998. 1044.]
 [ 136.  122.  160.  109.  162.  452.  920.  973. 1013. 1029.]
 [ 108.  108.  161.  142.  204.  606. 1003. 1039. 1081. 1067.]
 [ 200.  228.  190.  206.  417.  911. 1046. 1064. 1089. 1122.]
 [ 230.  217.  165.  227.  617. 1055. 1067. 1066. 1080. 1099.]
 [ 238.  189.  165.  308.  842. 1103. 1084. 1084. 1087. 1101.]
 [ 182.  156.  200.  528.  966. 1082. 1076. 1099. 1092. 1062.]
 [ 241.  238.  487.  818. 1011. 1050. 1091. 1103. 1090. 1069.]]
<class 'numpy.ndarray'>
float64


**Моделирование преобразования 3D массива в 2D и обратно**

In [ ]:
D3 = np.array([[[1, 2, 3, 4],  # создадим не пустой 3D массив
        [0, 0, 0, 0],
        [0, 0, 0, 0],
        [21, 22, 23, 24]]])
print('Исходный массив D3','\n', D3)
D3_D2 = np.reshape(D3,(1,16))     # изменим размерность в 2D
print('Преобразованный в D2', D3_D2)                             # смотрим
D2_D3 = np.reshape(D3_D2,(1,4,4)) # сделаем обратное преобразование размерности 2D в 3D
print('Восстановленный D3','\n', D2_D3)

Исходный массив D3 
 [[[ 1  2  3  4]
  [ 0  0  0  0]
  [ 0  0  0  0]
  [21 22 23 24]]]
Преобразованный в D2 [[ 1  2  3  4  0  0  0  0  0  0  0  0 21 22 23 24]]
Восстановленный D3 
 [[[ 1  2  3  4]
  [ 0  0  0  0]
  [ 0  0  0  0]
  [21 22 23 24]]]


array([[ 1,  2,  3,  4,  5,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0, 21, 22, 23, 24, 25]])

In [ ]:
D2_D3 = np.reshape(D3_D2,(1,5,5))
D2_D3

array([[[ 1,  2,  3,  4,  5],
        [ 0,  0,  0,  0,  0],
        [ 0,  0,  0,  0,  0],
        [ 0,  0,  0,  0,  0],
        [21, 22, 23, 24, 25]]])

# Все прочее

In [ ]:
# Чтения массива в NB colab из googl diska:

with open('/content/drive/MyDrive/Baza/Stag_1/np/LUNG1_092', 'r') as f:
  LUNG1_092 = np.loadtxt(f)
  print(type(LUNG1_092), LUNG1_092.shape)

### Запись и чтение массива на GooglDick


In [ ]:
a = np.array([1, 2, 3, 4, 5,6]) # создаем массив

with open('/content/drive/MyDrive/Baza/Stag_1/np/array.txt', 'w') as f: # Оператор with создает диспетчер контекста в Пайтоне,
                                                                        # который автоматически закрывает файл для вас, по окончанию работы в нем.
  np.savetxt(f, a)


In [ ]:
# Чтения массива в NB colab d gthtvtyye. np_array:

with open('/content/drive/MyDrive/Baza/Stag_1/np/array.txt', 'r') as f:
  a = np.loadtxt(f)
  print('a', a, type(a), a.shape)

a [1. 2. 3. 4. 5. 6.] <class 'numpy.ndarray'> (6,)
